In [ ]:
import random
import numpy as np
from itertools import groupby
from decimal import Decimal
%run ./packer.ipynb

In [ ]:
def seleccionarHijos(mutados,evaluados,elite):
    valorinv = np.argsort(evaluados)
    valor = np.flip(valorinv)
    
    seleccionados=[]
    for i in range(elite):
        seleccionados.append(mutados[valor[i]])

    return seleccionados
def mutar(padres):
    hijos = []
    numhijos = 3 #Hijos por padre: nueva población: 6
    i=0
    for h in range(numhijos):
        for k in range(len(padres)):
            x=random.random()
            hijos.append(padres[k][:])
            if x<0.85:
                m = random.randint(0,len(hijos[0])-1)
                n = random.randint(0,len(hijos[0])-1)
                aux = hijos[i][m]
                hijos[i][m] = hijos[i][n]
                hijos[i][n] = aux   
            i = i+1

    return hijos

def ls1_setminus_ls2(ls1, ls2):
    ls2_set = set(ls2)
    return [item for item in ls1 if item not in ls2]

def remove_duplicates(ls):
    # groupby needs ls to be sorted first
    new = sorted(ls)
    return list(key for key,val in groupby(new))

def davis_xover(x, y, i, j):
    # Init
    z = []
    length = len(x)
    xover_sect = x[i:j+1] # [i, j + 1) = [i, j]
    # Main
    fillers = ls1_setminus_ls2(y, xover_sect)
    for at in range(length):
        # copy xover section
        if i <= at <= j:
            z.insert(at, x[at])
        # use fillers
        else:
            filler = fillers.pop(0) # pop returns 1st elem and deletes it.
            z.insert(at, filler)
    return z


def combinar_Padres(poblacion,elite): #Aaaahhhhhh
    i = random.randint(0,len(poblacion)//4)
    aux = len(poblacion[0])//2
    j = i + aux # - 1 porque en davis_xover es j+1
    #x=len(poblacion)//2
    numPadres=elite
    padre = poblacion
    x = poblacion[0]
    y = poblacion[1]
    P1 = davis_xover(x, y, i, j)
    P2 = davis_xover(y, x, i, j)
   
    for i in range(numPadres//2):
        x = poblacion[2*i]
        y = poblacion[2*i+1]
        padre[2*i] = davis_xover(x, y, i, j)
        padre[2*i+1] = davis_xover(y, x, i, j)
    return padre[0:numPadres]

def seleccionar(poblacion,evaluada,padres):
    seleccionados = []
    acumulada = [evaluada[0]]
    indice= []
    for i in range(len(poblacion)-1):
        acumulada.append(evaluada[i+1]+acumulada[i])

    if sum(acumulada)==0:
        seleccionados = poblacion
        #Randomchoice
    else:
        for n in range(padres):
            x = random.random()
            for i in range(len(poblacion)):
                if x < acumulada[i]:
                    indice.append(i)
                    break
        for n in range(padres):
            seleccionados.append(poblacion[indice[n]])


    return seleccionados[1:padres]

def Evaluar(poblacion):
    valor = []
    valorNorm = []
    for i in range(len(poblacion)):
        cargarCajas()
        ev = packer.pack(poblacion[i])
        valor.append(ev.get_filling_ratio())
        packer.reset()
        #packer = Packer()
    for i in range(len(valor)):
        if not sum(valor)==0:
            valorNorm.append((valor[i]/sum(valor)))
        else:
            return valor
    return valorNorm
    #return valor

def generar_lista(tam):
    lista = [0] * tam
    for i in range(tam):
        lista[i] = i
    return lista

def generarPoblacion(cantidad,cromosomas):
    poblacion = []
    for _ in range(cromosomas):
        orden = generar_lista(cantidad)
        random.shuffle(orden)
        poblacion.append(orden)
    return poblacion

def Genetico(cantidad,cromosomas):
    poblacion = generarPoblacion(cantidad,cromosomas)
    numGeneraciones=10
    maxtotal = 0
    mejorSol = generarPoblacion(1,1)
    for g in range(numGeneraciones):
        #elite = 30
        poblacionEvaluada = Evaluar(poblacion)
        poblacionSeleccionada = seleccionar(poblacion,poblacionEvaluada,4)
        combinados = combinar_Padres(poblacionSeleccionada,2)
        mutados = mutar(combinados)
        hijosEvaluados = Evaluar(mutados)
        #print(hijosEvaluados)
        mejoresHijos = seleccionarHijos(mutados,hijosEvaluados,2)
        soluciones = []
        
        for i in range(len(mejoresHijos)):
            cargarCajas()
            ev = packer.pack(mejoresHijos[i])
            soluciones.append(ev.get_filling_ratio())
            packer.reset()
        print("Generación:",g)
        #print(max(soluciones))
        if max(soluciones)>maxtotal:
            maxtotal = max(soluciones)
            mejorind = soluciones.index(max(soluciones))
            mejorSol = mejoresHijos[mejorind]
        poblacion = mejoresHijos
        #evaluacion = packer.pack(poblacion[0])
        #print(evaluacion)
    return mejorSol


packer = Packer()




def cargarCajas():
    
    #packer.add_bin(Bin('large-2-box', 587, 233, 220, 50000000.0))
    #for _ in range(40):
    #    packer.add_item(Item('50g [powder 1]', 108, 76, 30, 1))
    #for _ in range(33):
    #    packer.add_item(Item('50g [powder 2]', 110, 43, 25, 2))
    #for _ in range(41):
    #    packer.add_item(Item('50g [powder 3]', 97, 81, 27, 3))
    #packer.add_bin(Bin('trailerbox', 13.62, 2.46, 2.70, 700.0))
    #packer.add_bin(Bin('large-box', 12.0, 12.0, 5.5, 70.0))
    #packer.add_bin(Bin('large-2-box', 23.6875, 11.75, 3.0, 70.0))
    packer.add_bin(Bin('trailerbox', 250, 100, 100, 700.0)) #palets 0.86
    
    for _ in range(20):
        packer.add_item(Item('Libros', 23, 14, 10, 1)) #vol = 3220
    for _ in range(25):
        packer.add_item(Item('Zapatos', 35, 20, 15, 1))#vol = 10500
    for _ in range(30):
        packer.add_item(Item('Mudanza', 75, 35, 35, 1))#vol = 91975
    
    
    
    return




#cantidad = len(packer.unplaced_items)
cantidad = 75
cromosomas = 4
sol = Genetico(cantidad,cromosomas)
print("El orden óptimo es:", sol)
packer.reset()
cargarCajas()
ev = packer.pack(sol)
print("Fitting ratio:",ev.get_filling_ratio())
print(ev.string())




::::::::::: small-envelope(11.500x6.125x0.250, max_weight:10.000) vol(17.609) item_number(0) filling_ratio(0.000)
FITTED ITEMS:

UNFITTED ITEMS:
====>  250g [powder 8](7.874x3.937x1.968, weight: 4.000) pos([0, 0, 0]) rt(5) vol(61.008)
====>  250g [powder 9](7.874x3.937x1.968, weight: 5.000) pos([0, 0, 0]) rt(5) vol(61.008)
====>  250g [powder 10](7.874x3.937x1.968, weight: 6.000) pos([0, 0, 0]) rt(5) vol(61.008)
====>  250g [powder 11](7.874x3.937x1.968, weight: 7.000) pos([0, 0, 0]) rt(5) vol(61.008)
====>  250g [powder 12](7.874x3.937x1.968, weight: 8.000) pos([0, 0, 0]) rt(5) vol(61.008)
====>  250g [powder 13](7.874x3.937x1.968, weight: 9.000) pos([0, 0, 0]) rt(5) vol(61.008)
====>  50g [powder 1](3.937x1.968x1.968, weight: 1.000) pos([0, 0, 0]) rt(5) vol(15.248)
====>  50g [powder 2](3.937x1.968x1.968, weight: 2.000) pos([0, 0, 0]) rt(5) vol(15.248)
====>  50g [powder 3](3.937x1.968x1.968, weight: 3.000) pos([0, 0, 0]) rt(5) vol(15.248)
====>  50g [powder 4](3.937x1.968x1.968, we